<a href="https://colab.research.google.com/github/annaouyang/open-hacks-2020/blob/master/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Pre-processing

The goal of this notebook is to join Google's mobility data (https://www.google.com/covid19/mobility/) with COVID-19 case counts on a county-by-county basis (United States only).


## About the Data

### 1. Google COVID-19 Community Reports 

According to their website, "the data shows how visitors to (or time spent in) categorized places change compared to our baseline days. A baseline day represents a normal value for that day of the week. The baseline day is the median value from the 5‑week period Jan 3 – Feb 6, 2020.

For each region-category, the baseline isn’t a single value—it’s 7 individual values. The same number of visitors on 2 different days of the week, result in different percentage changes. So, we recommend the following:

Don’t infer that larger changes mean more visitors or smaller changes mean less visitors. Avoid comparing day-to-day changes. Especially weekends with weekdays."

Citation: Google LLC "Google COVID-19 Community Mobility Reports".
https://www.google.com/covid19/mobility/ Accessed: May 23, 2020.


### 2. Population 

We anticipate that COVID-19 case counts will be closely related to population. We will be using 2019 population data for US counties provided here: https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-total.html.

We were unable to find reliable land area values for US counties. Thus, we were unable to calculate population density values.


### 3: COVID-19 case counts

We will be using the COVID-19 case counts provided by the New York Times in the following GitHub repo: https://github.com/nytimes/covid-19-data. Historical data is saved as the total # of COVID-19 cases for a county at the end of each day.

https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv



## Load the data

In [0]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import timedelta

In [2]:
# for now, pull from CSVs in repo
google_url = 'https://raw.githubusercontent.com/annaouyang/open-hacks-2020/master/google_mobility.csv'
population_url = 'https://raw.githubusercontent.com/annaouyang/open-hacks-2020/master/us-census-population.csv'


google_col = ['country_region_code', 'country_region', 'state','county', 'date', 'retail_and_rec', 'grocery_and_pharmacy','parks','transit_stations','workplaces','residential']

population_col = ['county', 'state', 'population']
population_dtype_lst = [str, str, int]
population_dtype_dict = dict(zip(population_col, population_dtype_lst))


google_df = pd.read_csv(google_url, names=google_col, skiprows=[0])
population_df = pd.read_csv(population_url, names=population_col, dtype=population_dtype_dict , skiprows=[0])

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
# get fips data

fips_url = 'https://raw.githubusercontent.com/annaouyang/open-hacks-2020/master/fips.csv'

fips_df = pd.read_csv(fips_url, dtype={'state_fips': str, 'county_fips': str, 'area_name': str})

## Preprocess FIPS data (state and county)

In [4]:
fips_df.head()

,state_fips,county_fips,area_name
0,01,000,Alabama
1,01,001,Autauga County
2,01,003,Baldwin County
3,01,005,Barbour County
4,01,007,Bibb County


In [5]:
state_fips = fips_df.drop_duplicates(subset=['state_fips'], keep='first')
state_fips = state_fips.drop(columns=['county_fips'])
state_fips.columns = ['state_fips', 'state']
state_fips.head()

,state_fips,state
0,01,Alabama
529,02,Alaska
707,04,Arizona
814,05,Arkansas
1392,06,California


In [14]:
county_fips = fips_df.merge(state_fips, on=['state_fips'])
county_fips = county_fips.loc[county_fips['county_fips'] != '000'] # if 000, is name of state
county_fips.columns = ['state_fips', 'county_fips', 'county', 'state']
county_fips.head()

,state_fips,county_fips,county,state
1,01,001,Autauga County,Alabama
2,01,003,Baldwin County,Alabama
3,01,005,Barbour County,Alabama
4,01,007,Bibb County,Alabama
5,01,009,Blount County,Alabama


In [141]:
county_fips[county_fips['county'] == 'Virginia Beach city']

,state_fips,county_fips,county,state
40302,51,810,Virginia Beach city,Virginia


## Preprocess Google Mobility Data

In [7]:
google_df.head()

,country_region_code,country_region,state,county,date,retail_and_rec,grocery_and_pharmacy,parks,transit_stations,workplaces,residential
0,AE,United Arab Emirates,NaN,NaN,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,AE,United Arab Emirates,NaN,NaN,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
2,AE,United Arab Emirates,NaN,NaN,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
3,AE,United Arab Emirates,NaN,NaN,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
4,AE,United Arab Emirates,NaN,NaN,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0


In [8]:
# change date to datetime
google_df['date'] = pd.to_datetime(google_df['date'])
google_df.dtypes

country_region_code             object
country_region                  object
state                           object
county                          object
date                    datetime64[ns]
retail_and_rec                 float64
grocery_and_pharmacy           float64
parks                          float64
transit_stations               float64
workplaces                     float64
residential                    float64
dtype: object

In [9]:
# only keep data for counties
df = google_df.loc[google_df['country_region_code'] == 'US'] 
df.head()

,country_region_code,country_region,state,county,date,retail_and_rec,grocery_and_pharmacy,parks,transit_stations,workplaces,residential
111798,US,United States,NaN,NaN,2020-02-15,6.0,2.0,15.0,3.0,2.0,-1.0
111799,US,United States,NaN,NaN,2020-02-16,7.0,1.0,16.0,2.0,0.0,-1.0
111800,US,United States,NaN,NaN,2020-02-17,6.0,0.0,28.0,-9.0,-24.0,5.0
111801,US,United States,NaN,NaN,2020-02-18,0.0,-1.0,6.0,1.0,0.0,1.0
111802,US,United States,NaN,NaN,2020-02-19,2.0,0.0,8.0,1.0,1.0,0.0


In [0]:
df = df.dropna(subset=['county']).reset_index()

In [0]:
df = df.drop(columns=['index','country_region_code', 'country_region'])

In [0]:
# get day of week
df['day_of_week'] = df['date'].dt.dayofweek

In [15]:
df.shape

(242747, 10)

In [0]:
# add county fips
df = df.merge(county_fips, how='left', on=['county','state'])

In [0]:
# check that virginia beach is 810

In [0]:
# check anchorage 020

In [0]:
# check st louis 

In [18]:
# reorder columns
df = df[['state', 'county', 'state_fips', 'county_fips','date', 'day_of_week', 'retail_and_rec', 'grocery_and_pharmacy',
       'parks', 'transit_stations', 'workplaces', 'residential']]
df.head()

,state,county,state_fips,county_fips,date,day_of_week,retail_and_rec,grocery_and_pharmacy,parks,transit_stations,workplaces,residential
0,Alabama,Autauga County,01,001,2020-02-15,5,5.0,7.0,NaN,NaN,-4.0,NaN
1,Alabama,Autauga County,01,001,2020-02-16,6,0.0,1.0,-23.0,NaN,-4.0,NaN
2,Alabama,Autauga County,01,001,2020-02-17,0,8.0,0.0,NaN,NaN,-27.0,5.0
3,Alabama,Autauga County,01,001,2020-02-18,1,-2.0,0.0,NaN,NaN,2.0,0.0
4,Alabama,Autauga County,01,001,2020-02-19,2,-2.0,0.0,NaN,NaN,2.0,0.0


In [19]:
df.describe()

,day_of_week,retail_and_rec,grocery_and_pharmacy,parks,transit_stations,workplaces,residential
count,242747.000000,178314.000000,170226.000000,59632.000000,91877.000000,232592.000000,107037.000000
mean,2.916666,-14.325241,0.602898,11.852613,-15.712170,-22.497003,9.407794
std,1.969375,23.687440,15.552082,44.537799,24.149703,18.588260,8.391851
min,0.000000,-100.000000,-81.000000,-91.000000,-91.000000,-81.000000,-11.000000
25%,1.000000,-33.000000,-9.000000,-17.000000,-32.000000,-36.000000,0.000000
50%,3.000000,-14.000000,1.000000,7.000000,-13.000000,-26.000000,10.000000
75%,5.000000,6.000000,9.000000,34.000000,2.000000,-2.000000,16.000000
max,6.000000,226.000000,174.000000,468.000000,222.000000,43.000000,38.000000


In [142]:
df[df['county'] == 'Virginia Beach']

,state,county,state_fips,county_fips,date,day_of_week,retail_and_rec,grocery_and_pharmacy,parks,transit_stations,workplaces,residential
226561,Virginia,Virginia Beach,NaN,NaN,2020-02-15,5,4.0,0.0,8.0,-3.0,1.0,0.0
226562,Virginia,Virginia Beach,NaN,NaN,2020-02-16,6,6.0,2.0,5.0,6.0,-1.0,-1.0
226563,Virginia,Virginia Beach,NaN,NaN,2020-02-17,0,11.0,3.0,59.0,-4.0,-31.0,7.0
226564,Virginia,Virginia Beach,NaN,NaN,2020-02-18,1,2.0,2.0,21.0,6.0,2.0,0.0
226565,Virginia,Virginia Beach,NaN,NaN,2020-02-19,2,2.0,1.0,1.0,-8.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
226648,Virginia,Virginia Beach,NaN,NaN,2020-05-12,1,-24.0,-1.0,46.0,-32.0,-42.0,17.0
226649,Virginia,Virginia Beach,NaN,NaN,2020-05-13,2,-22.0,0.0,53.0,-32.0,-43.0,16.0
226650,Virginia,Virginia Beach,NaN,NaN,2020-05-14,3,-24.0,-1.0,55.0,-31.0,-43.0,NaN
226651,Virginia,Virginia Beach,NaN,NaN,2020-05-15,4,-22.0,0.0,71.0,-33.0,-43.0,16.0


## Preprocess Population Data


In [20]:
population_df.head()

,county,state,population
0,Autauga County,Alabama,55869
1,Baldwin County,Alabama,223234
2,Barbour County,Alabama,24686
3,Bibb County,Alabama,22394
4,Blount County,Alabama,57826


In [21]:
population_df.dtypes

county        object
state         object
population     int64
dtype: object

## Preprocess Coronavirus Count Data

In [0]:
case_count_url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'
case_count_df = pd.read_csv(case_count_url, usecols=['fips', 'date', 'cases'], 
                            dtype={'fips': str,
                                   'date' : str,
                                   'cases': int})

In [62]:
case_count_df.head()

,date,fips,cases
0,2020-01-21,53061,1
1,2020-01-22,53061,1
2,2020-01-23,53061,1
3,2020-01-24,17031,1
4,2020-01-24,53061,1


In [63]:
case_count_df['date'] = pd.to_datetime(case_count_df['date'])
case_count_df.dtypes

date     datetime64[ns]
fips             object
cases             int64
dtype: object

In [0]:
# apply lambda function? 
def get_num_new_cases(df, row):
  fips = row.fips

  today = row.date
  today_count = row.cases

  yesterday = today - timedelta(days=1)
  yesterday_row = df.loc[(df['fips'] == fips) & (df['date'] == yesterday)]
  yesterday_count = 0

  if not yesterday_row.empty:
    yesterday_count = yesterday_row.cases
  
  return int(today_count - yesterday_count) # if first day, just today's count

In [0]:
test_5000 = case_count_df.head(5000)

In [110]:
# testing 500 = 6 seconds
print(dt.datetime.now())
test_500['new_cases'] = test_500.apply(lambda row: get_num_new_cases(case_count_df, row), axis=1)
print(dt.datetime.now())

2020-05-23 23:54:05.949273
2020-05-23 23:54:11.732445


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [114]:
# 5000 = 1 min
print(dt.datetime.now())
test_5000['new_cases'] = test_5000.apply(lambda row: get_num_new_cases(case_count_df, row), axis=1)
print(dt.datetime.now())

2020-05-23 23:55:04.822656
2020-05-23 23:56:01.794407


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [116]:
#10k = 2 min
print(dt.datetime.now())
test_10k = case_count_df.head(10000)
test_10k['new_cases'] = test_10k.apply(lambda row: get_num_new_cases(case_count_df, row), axis=1)
print(dt.datetime.now())

2020-05-23 23:58:17.610774
2020-05-24 00:00:11.955198


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [118]:
test_10k.head(20)

,date,fips,cases,new_cases
0,2020-01-21,53061,1,1
1,2020-01-22,53061,1,0
2,2020-01-23,53061,1,0
3,2020-01-24,17031,1,1
4,2020-01-24,53061,1,0
5,2020-01-25,06059,1,1
6,2020-01-25,17031,1,0
7,2020-01-25,53061,1,0
8,2020-01-26,04013,1,1
9,2020-01-26,06037,1,1


In [121]:
print(dt.datetime.now())
case_count_df['new_cases'] = case_count_df.apply(lambda row: get_num_new_cases(case_count_df, row), axis=1)
print(dt.datetime.now())

2020-05-24 00:03:42.325810
2020-05-24 00:36:09.076088


In [0]:
# save to csv
case_count_df.to_csv('cases_by_county.csv', index=False)
!cp cases_by_county.csv "/content/drive/My Drive/"

In [126]:
case_count_df.tail()

,date,fips,cases,new_cases
167834,2020-05-22,56035,3,0
167835,2020-05-22,56037,25,0
167836,2020-05-22,56039,100,0
167837,2020-05-22,56041,13,0
167838,2020-05-22,56043,19,0


### Try to join datasets on date, county, state

In [128]:
test = population_df.merge(df, how='right', on=['state', 'county'])
test.head()

,county,state,population,state_fips,county_fips,date,day_of_week,retail_and_rec,grocery_and_pharmacy,parks,transit_stations,workplaces,residential
0,Autauga County,Alabama,55869,01,001,2020-02-15,5,5.0,7.0,NaN,NaN,-4.0,NaN
1,Autauga County,Alabama,55869,01,001,2020-02-16,6,0.0,1.0,-23.0,NaN,-4.0,NaN
2,Autauga County,Alabama,55869,01,001,2020-02-17,0,8.0,0.0,NaN,NaN,-27.0,5.0
3,Autauga County,Alabama,55869,01,001,2020-02-18,1,-2.0,0.0,NaN,NaN,2.0,0.0
4,Autauga County,Alabama,55869,01,001,2020-02-19,2,-2.0,0.0,NaN,NaN,2.0,0.0


In [129]:
# verify that join was successful
test[test['population'].isnull()].drop_duplicates(subset=['county', 'state'])

,county,state,population,state_fips,county_fips,date,day_of_week,retail_and_rec,grocery_and_pharmacy,parks,transit_stations,workplaces,residential


In [0]:
# let fips be full 5 digit code
test['fips'] = test['state_fips'] + test['county_fips']

In [152]:
# merge population, google mobility data, and covid cases data
final_df = test.merge(case_count_df, on=['date','fips'], how='left')
final_df.head()

,county,state,population,state_fips,county_fips,date,day_of_week,retail_and_rec,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,fips,cases,new_cases
0,Autauga County,Alabama,55869,01,001,2020-02-15,5,5.0,7.0,NaN,NaN,-4.0,NaN,01001,NaN,NaN
1,Autauga County,Alabama,55869,01,001,2020-02-16,6,0.0,1.0,-23.0,NaN,-4.0,NaN,01001,NaN,NaN
2,Autauga County,Alabama,55869,01,001,2020-02-17,0,8.0,0.0,NaN,NaN,-27.0,5.0,01001,NaN,NaN
3,Autauga County,Alabama,55869,01,001,2020-02-18,1,-2.0,0.0,NaN,NaN,2.0,0.0,01001,NaN,NaN
4,Autauga County,Alabama,55869,01,001,2020-02-19,2,-2.0,0.0,NaN,NaN,2.0,0.0,01001,NaN,NaN


In [160]:
final_df.new_cases.shape[0] - final_df.new_cases.isna().sum()

211962

In [0]:
# issues with virginia and alaska 'cities' and 'municipalities'
final_df[final_df['fips'].isnull()].drop_duplicates(subset=['county', 'state'])

In [156]:
# export final df to csv

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:

final_df.to_csv('mobility_and_case_count.csv', index=False)
!cp mobility_and_case_count.csv "/content/drive/My Drive/"